In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchsummary import summary

In [14]:
torch.cuda.empty_cache()

In [15]:
a = torch.rand([32,3,224,224])
a = nn.Sequential(nn.Conv2d(3,64,7,2,3),
                    nn.BatchNorm2d(64),nn.ReLU(),
                    nn.MaxPool2d(3,2,1))(a)
a.shape

torch.Size([32, 64, 56, 56])

In [16]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)).cuda()])
trainset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data',train=True,download=False,transform=transform)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True,num_workers=2)
testset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data',train=False,download=False,transform=transform)
test_loader = torch.utils.data.DataLoader(testset,batch_size=1000,shuffle=True,num_workers=2)

In [17]:
class ResNeXt(nn.Module):
    
    def __init__(self) -> None:
        
        super().__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(3,64,7,2,3),
                                   nn.BatchNorm2d(64),nn.ReLU(),
                                   nn.MaxPool2d(3,2,1))
        
        self.conv2d_x = nn.Sequential(nn.Conv2d(64,128,1,1),nn.BatchNorm2d(128),nn.ReLU(),
                                   nn.Conv2d(128,128,3,1,1,groups=32),nn.BatchNorm2d(128),nn.ReLU(),
                                   nn.Conv2d(128,256,1,1),nn.BatchNorm2d(256),nn.ReLU())
        self.conv2_x = nn.Sequential(nn.Conv2d(256,128,1,1),nn.BatchNorm2d(128),nn.ReLU(),
                                   nn.Conv2d(128,128,3,1,1,groups=32),nn.BatchNorm2d(128),nn.ReLU(),
                                   nn.Conv2d(128,256,1,1),nn.BatchNorm2d(256),nn.ReLU())
        
        self.conv3d_x = nn.Sequential(nn.Conv2d(256,256,1,1),nn.BatchNorm2d(256),nn.ReLU(),
                                   nn.Conv2d(256,256,3,2,1,groups=32),nn.BatchNorm2d(256),nn.ReLU(),
                                   nn.Conv2d(256,512,1,1),nn.BatchNorm2d(512),nn.ReLU())
        self.conv3_x = nn.Sequential(nn.Conv2d(512,256,1,1),nn.BatchNorm2d(256),nn.ReLU(),
                                   nn.Conv2d(256,256,3,1,1,groups=32),nn.BatchNorm2d(256),nn.ReLU(),
                                   nn.Conv2d(256,512,1,1),nn.BatchNorm2d(512),nn.ReLU())
        
        self.conv4d_x = nn.Sequential(nn.Conv2d(512,512,1,1),nn.BatchNorm2d(512),nn.ReLU(),
                                   nn.Conv2d(512,512,3,2,1,groups=32),nn.BatchNorm2d(512),nn.ReLU(),
                                   nn.Conv2d(512,1024,1,1),nn.BatchNorm2d(1024),nn.ReLU())
        self.conv4_x = nn.Sequential(nn.Conv2d(1024,512,1,1),nn.BatchNorm2d(512),nn.ReLU(),
                                   nn.Conv2d(512,512,3,1,1,groups=32),nn.BatchNorm2d(512),nn.ReLU(),
                                   nn.Conv2d(512,1024,1,1),nn.BatchNorm2d(1024),nn.ReLU())
        
        self.conv5d_x = nn.Sequential(nn.Conv2d(1024,1024,1,1),nn.BatchNorm2d(1024),nn.ReLU(),
                                   nn.Conv2d(1024,1024,3,2,1,groups=32),nn.BatchNorm2d(1024),nn.ReLU(),
                                   nn.Conv2d(1024,2048,1,1),nn.BatchNorm2d(2048),nn.ReLU())
        self.conv5_x = nn.Sequential(nn.Conv2d(2048,1024,1,1),nn.BatchNorm2d(1024),nn.ReLU(),
                                   nn.Conv2d(1024,1024,3,1,1,groups=32),nn.BatchNorm2d(1024),nn.ReLU(),
                                   nn.Conv2d(1024,2048,1,1),nn.BatchNorm2d(2048),nn.ReLU())
        
        self.avgpool = nn.AvgPool2d(1,1)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(2048,1000)
        
    def forward(self,x):
        
        x = self.conv1(x)
        x1 = self.conv2d_x(x)
        x1 = self.conv2_x(x1)
        x = nn.Conv2d(64,256,1,1).cuda()(x)
        x = torch.add(x,x1).cuda()
        
        x1 = self.conv2_x(x)
        x2 = self.conv3d_x(x1)
        x1 = nn.Conv2d(256,512,1,1).cuda()(x)
        x1 = nn.MaxPool2d(3,2,1).cuda()(x1)
        x2 = torch.add(x1,x2).cuda()
        
        x3 = self.conv3_x(x2)
        x3 = self.conv3_x(x3)
        x3 = torch.add(x2,x3).cuda()
        
        x4 = self.conv3_x(x3)
        x5 = self.conv4d_x(x4)
        x3 = nn.MaxPool2d(3,2,1).cuda()(x3)
        x3 = nn.Conv2d(512,1024,1,1).cuda()(x3)
        x5 = torch.add(x3,x5).cuda()
        
        x6 = self.conv4_x(x5)
        x6 = self.conv4_x(x6)
        x5 = torch.add(x5,x6)
        
        x6 = self.conv4_x(x5)
        x6 = self.conv4_x(x6)
        x5 = torch.add(x5,x6).cuda()
        
        x6 = self.conv4_x(x5)
        x7 = self.conv5d_x(x6)
        x6 = nn.MaxPool2d(3,2,1).cuda()(x5)
        x6 = nn.Conv2d(1024,2048,1,1).cuda()(x6)
        x7 = torch.add(x6,x7).cuda()
        
        x8 = self.conv5_x(x7)
        x8 = self.conv5_x(x8)
        x8 = torch.add(x7,x8).cuda()

        x8 = self.avgpool(x8)
        x8 = nn.Flatten().cuda()(x8)
        x8 = self.linear(x8)
        
        return x8

In [18]:
model = ResNeXt().cuda()
try:
    model.load_state_dict(torch.load('C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\Neural Networks\\Pytorch implementation\\All Models Saved\\ResNext50.pth'))
except:
    print("Couldn\'t find a saved weights of the model")

Couldn't find a saved weights of the model


In [19]:
summary(model=model,input_size=(3,32,32),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 64, 16, 16]           9,472
       BatchNorm2d-2           [64, 64, 16, 16]             128
              ReLU-3           [64, 64, 16, 16]               0
         MaxPool2d-4             [64, 64, 8, 8]               0
            Conv2d-5            [64, 128, 8, 8]           8,320
       BatchNorm2d-6            [64, 128, 8, 8]             256
              ReLU-7            [64, 128, 8, 8]               0
            Conv2d-8            [64, 128, 8, 8]           4,736
       BatchNorm2d-9            [64, 128, 8, 8]             256
             ReLU-10            [64, 128, 8, 8]               0
           Conv2d-11            [64, 256, 8, 8]          33,024
      BatchNorm2d-12            [64, 256, 8, 8]             512
             ReLU-13            [64, 256, 8, 8]               0
           Conv2d-14            [64, 12

In [20]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=1e-3,momentum=0.9,weight_decay=0.0001)
lr_scheduler = optim.lr_scheduler.CyclicLR(optimizer=optimizer,
                                           base_lr=1e-3,
                                           max_lr=1e-1,
                                           step_size_up=5,
                                           step_size_down=5,
                                           cycle_momentum=False,
                                           mode="exp_range")

'lr_scheduler = optim.lr_scheduler.CyclicLR(optimizer=optimizer,\n                                           base_lr=1e-5,\n                                           max_lr=1e-2,\n                                           step_size_up=5,\n                                           step_size_down=5,\n                                           cycle_momentum=False,\n                                           mode="exp_range")'

In [21]:
loss_every_epoch = []

def test():
    
    model.eval()
    total = 0.0
    accuracy = 0.0
    
    for i,(inputs,labels) in enumerate(test_loader,0):
            
        inputs,labels = inputs.cuda(),labels.cuda()
        ouputs = model(inputs)
        _, prediction = torch.max(ouputs.data,1)
        total += labels.size(0)
        accuracy += (prediction == labels).sum().item()
    
    print("Accuracy of images {}/{}".format(accuracy,total))
    print("Test accuracy of this epoch is : {}".format((accuracy/total)*100))
    
        
def train(epoch):
    for ep in range(0,epoch):
        losses = 0.0
        
        for i,(inputs,labels) in enumerate(train_loader,0):
            
            inputs,labels = inputs.cuda(),labels.cuda()
            optimizer.zero_grad(set_to_none=False)
            outputs = model(inputs)
            loss = loss_func(outputs,labels)
            losses += loss.item()
            loss.backward()
            optimizer.step()
        
        lr_scheduler.step()
        print("Training loss for the Epoch is {} : {}".format(ep+1,losses/i+1))
        loss_info = "Training loss for Epoch "+str(ep+1)+" is "+str((losses/i+1))
        loss_every_epoch.append(loss_info)
        test()

In [22]:
train(25)

Training loss for the Epoch is 1 : 6.325426419168978
Accuracy of images 106.0/10000.0
Test accuracy of this epoch is : 1.06
Training loss for the Epoch is 2 : 5.569156669318752
Accuracy of images 260.0/10000.0
Test accuracy of this epoch is : 2.6
Training loss for the Epoch is 3 : 5.340376798077826
Accuracy of images 448.0/10000.0
Test accuracy of this epoch is : 4.4799999999999995
Training loss for the Epoch is 4 : 5.160644457679094
Accuracy of images 606.0/10000.0
Test accuracy of this epoch is : 6.0600000000000005
Training loss for the Epoch is 5 : 5.050976864049132
Accuracy of images 807.0/10000.0
Test accuracy of this epoch is : 8.07
Training loss for the Epoch is 6 : 4.96563289473823
Accuracy of images 861.0/10000.0
Test accuracy of this epoch is : 8.61
Training loss for the Epoch is 7 : 4.8680844316104
Accuracy of images 916.0/10000.0
Test accuracy of this epoch is : 9.16
Training loss for the Epoch is 8 : 4.782733306689391
Accuracy of images 1096.0/10000.0
Test accuracy of this

In [23]:
torch.save(model.state_dict(), 'C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\Neural Networks\\Pytorch implementation\\All Models Saved\\ResNext50.pth')